In [1]:
dir = "/home/jim/Playing/cf/abair-corpora/"
col_names = ["id", "set", "gender", "url", "text", "phonemes"]

In [2]:
import os

In [3]:
inputs = []
for (path, dirs, files) in os.walk(dir):
    if "/comhra/" in path:
        continue
    if "corpusfile.txt" in files:
        inputs.append(os.path.join(path, "corpusfile.txt"))

In [ ]:
from datasets import load_dataset
dset = load_dataset("csv", data_files=inputs, column_names=col_names, delimiter="\t", split="train", index_col=False)
dset = dset.filter(lambda x: x['text'] is not None and "&" not in x['text'] and "2" not in x['text'])

In [5]:
def irish_lc(text):
    def lcword(word):
        if word[0:1] in "nt" and word[1:2] in "AÁEÉIÍOÓUÚ":
            return word[0:1] + "-" + word[1:].lower()
        else:
            return word.lower()
    return " ".join(list(map(lcword, text.split(" "))))

In [6]:
assert irish_lc("bhúr nAthair") == "bhúr n-athair"

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”…—]'

def remove_special_characters(batch):
    batch["text"] = batch["text"].replace("’", "'")
    batch["text"] = batch["text"].replace("(tor@m)", "")
    batch["text"] = re.sub(chars_to_ignore_regex, '', irish_lc(batch["text"])) + " "
    batch["text"] = batch["text"].replace("\xa0", " ")
    batch["text"] = batch["text"].replace("\ufeff", "")
    batch["text"] = batch["text"].replace("1983", "naoi déag ochtó a trí")
    batch["text"] = batch["text"].replace("tg4", "t g ceathair")
    return batch

dset = dset.map(remove_special_characters)

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = dset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dset.column_names)

In [ ]:
#vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))
vocab_list = list(set(vocab_train["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
print(vocab_dict)